In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [7]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [8]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [9]:
path = Path('../Resources/finished_data_dict.pkl')
data_dict_import = load_obj(path)

In [10]:
path = Path('../Resources/finished_key_list.csv')
key_list_import_df = pd.read_csv(path,index_col=0)
len(key_list_import_df)

9992

In [14]:
key_list = []
for i in key_list_import_df['Symbol']:
    key_list.append(i)
len(key_list)

9992

In [17]:
## Make a function to run this kind of model 

def mean_squared(symbol,validation_split=0.3,epochs=800,layers=1):
    index_value = 0
    for i in range(len(key_list)):
        if key_list[i] == symbol:
            index_value = i
            pass
    dict_key = key_list[index_value]
    df = data_dict_import[dict_key]['dataFrame'].copy()
    X = df.drop(columns='close').values
    y = y = df['close'].values
    
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)    
    
    num_of_inputs = 17
    num_of_outputs= 1

    
    ## Create Neural Network 
    if layers == 1:
        # Create a shallow, 1 hidden layer, neural network
        nn1 = Sequential()

        # Hidden layer
        nn1.add(Dense(units=8, input_dim=num_of_inputs, activation="relu"))

        # Output layer
        nn1.add(Dense(units=num_of_outputs, activation="linear"))

        # Compile the model
        nn1.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        # Fit the model
        model_1 = nn1.fit(X, y, validation_split=validation_split, epochs=epochs,verbose=0)
        
        # Save model_1 as JSON
        nn_json1 = nn1.to_json()

        file_path = Path("../Resources/model_1_temp.json")
        with open(file_path, "w") as json_file:
            json_file.write(nn_json1)

        # Save weights
        file_path = "../Resources/model_1_temp.h5"
        nn1.save_weights(file_path)
        
        # load json and create model
        file_path = Path("../Resources/model_1_temp.json")
        with open(file_path, "r") as json_file:
            model_json = json_file.read()
        loaded_model1 = model_from_json(model_json)

        # load weights into new model
        file_path = "../Resources/model_1_temp.h5"
        loaded_model1.load_weights(file_path)
        
        # Make some predictions with the loaded model
        df1 = df.copy()
        df1["predicted"] = loaded_model1.predict(X)
        df_pred = df1[['close','predicted']]
        df_pred
        
    elif layers == 2:
        # Define the model - deep neural network with two layers
        nn2 = Sequential()

        # First hidden layer
        nn2.add(Dense(units=8, input_dim=num_of_inputs, activation="relu"))

        # Second hidden layer
        nn2.add(Dense(units=8, activation="relu"))

        # Output layer
        nn2.add(Dense(units=num_of_outputs, activation="linear"))
        
        # Compile the model
        nn2.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

        # Fit the model
        model_2 = nn2.fit(X, y, validation_split=0.3, epochs=800, verbose=0)
        
        # Save model_2 as JSON
        nn_json2 = nn2.to_json()

        file_path = Path("../Resources/model_2_temp.json")
        with open(file_path, "w") as json_file:
            json_file.write(nn_json2)

        # Save weights
        file_path = "../Resources/model_2_temp.h5"
        nn2.save_weights(file_path)
        
        # load json and create model
        file_path = Path("../Resources/model_2_temp.json")
        with open(file_path, "r") as json_file:
            model_json = json_file.read()
        loaded_model2 = model_from_json(model_json)

        # load weights into new model
        file_path = "../Resources/model_2_temp.h5"
        loaded_model2.load_weights(file_path)
        
        # Make some predictions with the loaded model
        df2 = df.copy()
        df2["predicted"] = loaded_model2.predict(X)
        df_pred = df2[['close','predicted']]
        df_pred
        
    return df_pred

In [20]:
test_df = mean_squared('GME',layers=2)
test_df

,close,predicted
Date,,
2016-01-04,28.309999,28.433571
2016-01-05,28.770000,28.739128
2016-01-06,28.370001,28.286135
2016-01-07,28.450001,28.497614
2016-01-08,28.370001,28.358156
...,...,...
2021-10-25,173.970000,174.511322
2021-10-26,177.840000,180.752808
2021-10-27,173.510000,174.939011


In [21]:
test_df2 = mean_squared('TSLA',layers=2)
test_df2

,close,predicted
Date,,
2016-01-04,44.681999,44.514618
2016-01-05,44.686001,44.466122
2016-01-06,43.807999,43.602047
2016-01-07,43.130001,43.184448
2016-01-08,42.200001,42.301998
...,...,...
2021-10-25,1024.860000,160.896744
2021-10-26,1018.430000,334.912048
2021-10-27,1037.860000,348.723053
